In [49]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import torchvision.datasets as datasets
from scipy.io import loadmat

import matplotlib.pyplot as plt
import pandas as pd


In [50]:
class FlowerClassifier(nn.Module):
    def __init__(self, num_classes=102):
        super(FlowerClassifier,self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2 ,stride=2)
        self.fc1 = None
        self.fc2 = nn.Linear(512, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        
        flattened_size = x.size(1) * x.size(2) * x.size(3)
        self.fc1 = nn.Linear(flattened_size, 512)
        
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)

        return x

In [51]:
model = FlowerClassifier()
x = torch.randn(32,3, 112,112)
y = model(x)
print(y.shape)

torch.Size([32, 102])


In [52]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [53]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
train_dataset = datasets.Flowers102(root='./data', split='train', transform=transform, download=True)
val_dataset = datasets.Flowers102(root='./data', split='val', transform=transform, download=True)
test_dataset = datasets.Flowers102(root='./data', split='test', transform=transform, download=True)

# Create data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    print("Epoch [" + str(epoch + 1) + "/" + str(num_epochs) + "]")
    print("Loss: " + str(running_loss / len(train_loader)))
    print("Accuracy: " + str(100 * correct / total) + "%")

Epoch [1/10]
Loss: 4.6310229897499084
Accuracy: 0.8823529411764706%
Epoch [2/10]
Loss: 4.625406801700592
Accuracy: 1.1764705882352942%
Epoch [3/10]
Loss: 4.622036188840866
Accuracy: 0.7843137254901961%
Epoch [4/10]
Loss: 4.595176488161087
Accuracy: 0.7843137254901961%
Epoch [5/10]
Loss: 4.541414976119995
Accuracy: 1.1764705882352942%
